In [ ]:
#!pip install music21
!pip install seaborn

In [61]:
#Importing Libraries
import tensorflow 
import numpy as np 
import pandas as pd 
from collections import Counter
import random
import IPython
from IPython.display import Image, Audio
import music21
from music21 import *
import matplotlib.pyplot as plt 
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
import tensorflow.keras.backend as K
from tensorflow.keras.optimizers import Adamax
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
%matplotlib inline
import sys
import warnings
warnings.filterwarnings("ignore")
warnings.simplefilter("ignore")
np.random.seed(42)

In [64]:
#Loading the list of midi files as stream
filepath = "input/lofi-weights/"
import os, glob
#Getting midi files
all_midis = []
midi_files = glob.glob(os.path.join(filepath, "*.mid"))
if not midi_files:
    raise FileNotFoundError(f"No MIDI files found in {filepath}. Make sure the dataset is extracted there.")
for tr in midi_files:
    midi = converter.parse(tr)
    all_midis.append(midi)
    
#this will take time to load

print(all_midis)


[<music21.stream.Score 0x71c349c2f0e0>, <music21.stream.Score 0x71c34a102210>, <music21.stream.Score 0x71c349c2f740>, <music21.stream.Score 0x71c349e1df40>, <music21.stream.Score 0x71c349b96210>, <music21.stream.Score 0x71c349c53d40>, <music21.stream.Score 0x71c349c52b40>, <music21.stream.Score 0x71c349c6cc80>, <music21.stream.Score 0x71c349c6ffb0>, <music21.stream.Score 0x71c349c6e4b0>, <music21.stream.Score 0x71c349a3e480>, <music21.stream.Score 0x71c349cbb680>, <music21.stream.Score 0x71c349a59340>, <music21.stream.Score 0x71c3499a24b0>, <music21.stream.Score 0x71c349cbb8c0>, <music21.stream.Score 0x71c3499e35f0>, <music21.stream.Score 0x71c3498239b0>, <music21.stream.Score 0x71c349c2eae0>, <music21.stream.Score 0x71c349a3e060>, <music21.stream.Score 0x71c34989ee10>, <music21.stream.Score 0x71c3498eb440>, <music21.stream.Score 0x71c3498eb410>, <music21.stream.Score 0x71c349c2f4a0>, <music21.stream.Score 0x71c3497255e0>, <music21.stream.Score 0x71c34975d280>, <music21.stream.Score 0x

In [65]:
len(all_midis)

93

In [66]:

#function defination to get the notes.
    
def extract_notes(file):
    notes = []
    pick = None
    for j in file:
        songs = instrument.partitionByInstrument(j)
        for part in songs.parts:
            pick = part.recurse()
            for element in pick:
                if isinstance(element, note.Note):
                    notes.append(str(element.pitch))
                elif isinstance(element, chord.Chord):
                    notes.append(".".join(str(n) for n in element.normalOrder))

    return notes
#Getting the list of notes as Corpus
Corpus= extract_notes(all_midis)
print("Total notes in all the midi", len(Corpus))
print("Notes of the MIDI file",Corpus[:10])
print("Now as you see we have our music notes/chords in the corpus format.")

Total notes in all the midi 2046
Notes of the MIDI file ['C#5', 'B-4', 'C#5', '10.1', '10.1', 'B-4', 'C#5', '10.1', 'B-4', 'C#5']
Now as you see we have our music notes/chords in the corpus format.


In [70]:
# LilyPond install (commented out; sudo not available in this environment)
# !sudo apt-get -y install lilypond


In [73]:

#functions to print the MIDI notes without LilyPond
LILYPOND_AVAILABLE = False

def show(music):
    """Display a text summary and save a MIDI preview instead of using LilyPond."""
    try:
        print(music.show('text'))
    except Exception as exc:
        print(f"Text rendering failed: {exc}")
    try:
        music.write('midi', fp='preview.mid')
        print("Saved preview.mid (MIDI)")
    except Exception as exc:
        print(f"Could not save MIDI preview: {exc}")
    

def chords_n_notes(Snippet):
    Melody = []
    offset = 0 #Incremental
    for i in Snippet:
        #If it is chord
        if ("." in i or i.isdigit()):
            chord_notes = i.split(".") #Seperating the notes in chord
            notes = [] 
            for j in chord_notes:
                inst_note=int(j)
                note_snip = note.Note(inst_note)            
                notes.append(note_snip)
                chord_snip = chord.Chord(notes)
                chord_snip.offset = offset
                Melody.append(chord_snip)
        # pattern is a note
        else: 
            note_snip = note.Note(i)
            note_snip.offset = offset
            Melody.append(note_snip)
        # increase offset each iteration so that notes do not stack
        offset += 1
    Melody_midi = stream.Stream(Melody)   
    return Melody_midi

Melody_Snippet = chords_n_notes(Corpus[:100])
show(Melody_Snippet)


{0.0} <music21.note.Note C#>
{1.0} <music21.note.Note B->
{2.0} <music21.note.Note C#>
{3.0} <music21.chord.Chord B->
{3.0} <music21.chord.Chord B- C#>
{4.0} <music21.chord.Chord B->
{4.0} <music21.chord.Chord B- C#>
{5.0} <music21.note.Note B->
{6.0} <music21.note.Note C#>
{7.0} <music21.chord.Chord B->
{7.0} <music21.chord.Chord B- C#>
{8.0} <music21.note.Note B->
{9.0} <music21.note.Note C#>
{10.0} <music21.chord.Chord B->
{10.0} <music21.chord.Chord B- C#>
{11.0} <music21.note.Note B->
{12.0} <music21.note.Note C#>
{13.0} <music21.note.Note B->
{14.0} <music21.note.Note C#>
{15.0} <music21.note.Note B->
{16.0} <music21.note.Note C#>
{17.0} <music21.chord.Chord B->
{17.0} <music21.chord.Chord B- C#>
{18.0} <music21.note.Note B->
{19.0} <music21.chord.Chord B->
{19.0} <music21.chord.Chord B- C#>
{20.0} <music21.note.Note C#>
{21.0} <music21.note.Note B->
{22.0} <music21.note.Note C#>
{23.0} <music21.chord.Chord B->
{23.0} <music21.chord.Chord B- C#>
{24.0} <music21.note.Note B->
{25.

In [75]:
### Convert MIDI to MP3 (requires local audio tools)
- Needs either `timidity` **or** `fluidsynth` + a SoundFont (`SF2_PATH` env var), and `ffmpeg`.
- Uses `preview.mid` if present, otherwise `lofi_output.mid`.
- Runs conversion and plays the MP3 inline when the tools are available.

SyntaxError: invalid syntax (2397333697.py, line 2)

In [76]:
import os, subprocess, shutil
import IPython.display as ipd

# Choose a MIDI file to convert
midi_candidates = ["preview.mid", "lofi_output.mid"]
midi_file = next((m for m in midi_candidates if os.path.exists(m)), None)
if not midi_file:
    raise FileNotFoundError("No MIDI found (preview.mid or lofi_output.mid). Generate one first.")

# Find a renderer
renderer = next((b for b in ("timidity", "fluidsynth") if shutil.which(b)), None)
if renderer is None:
    raise EnvironmentError(
        "No MIDI renderer found (timidity or fluidsynth). Install one locally and rerun."
    )

wav_path = "preview.wav"
mp3_path = "preview.mp3"

if renderer == "timidity":
    subprocess.run(["timidity", midi_file, "-Ow", "-o", wav_path], check=True)
else:
    sf2 = os.environ.get("SF2_PATH")
    if not sf2 or not os.path.exists(sf2):
        raise EnvironmentError("Set SF2_PATH to a SoundFont (.sf2) for fluidsynth.")
    subprocess.run(["fluidsynth", "-ni", sf2, midi_file, "-F", wav_path, "-r", "44100"], check=True)

subprocess.run(["ffmpeg", "-y", "-i", wav_path, mp3_path], check=True)

display(ipd.Audio(mp3_path))
print(f"Saved {mp3_path}")


OSError: No MIDI renderer found (timidity or fluidsynth). Install one locally and rerun.

### Offline or Colab conversion (no local renderers)
- Download the MIDI and convert to MP3 with any DAW/online tool.
- Or run in Google Colab (has sudo) with:
  - `!apt-get -y install timidity ffmpeg`
  - `!timidity preview.mid -Ow -o preview.wav`
  - `!ffmpeg -y -i preview.wav preview.mp3`
- You can also pull the MIDI from this runtime and process elsewhere.

In [ ]:
from IPython.display import FileLink, display

# Pick a MIDI to download
midi_candidates = ["preview.mid", "lofi_output.mid"]
midi_file = next((m for m in midi_candidates if os.path.exists(m)), None)
if not midi_file:
    raise FileNotFoundError("No MIDI found (preview.mid or lofi_output.mid). Generate one first.")

print(f"Download this MIDI and convert to MP3 elsewhere: {midi_file}")
display(FileLink(midi_file))
